In [ ]:
import re
from enum import Enum
from struct import unpack
from hexdump import hexdump
from pathlib import Path
import bincopy
DUMPS = Path("dumps/")
DUMPS.mkdir(exist_ok=True)

FUL_FILE = DUMPS/Path("pinot_pp_PNP1CN2050BR_nbx_signed.ful")
DECOMPRESSED_BUFFER = DUMPS/Path("DecompressedBuffer.bin")
BOOTLOADER_SREC = DUMPS/Path("BootLoader.srec")
BINARY_SREC = DUMPS/Path("BinarySrec.srec")
RAW_FLASH_BIN = DUMPS/Path("RawFlash.bin")
FLASH_BIN = DUMPS/Path("Flash.bin")
CLEANED_BOOTLOADER_SREC = DUMPS/Path("CleanedBootLoader.srec")

BOOTLOADER_DUMPS = DUMPS/"bootloader"
BOOTLOADER_DUMPS.mkdir(exist_ok=True)


In [ ]:
PAGE_SIZE = 2048
OOB_SIZE = 0x40


class CompressionMethod(Enum):
    UNENCODED = 0
    RLE = 1
    TIFF = 2
    DELTA_ROW = 3
    RESERVED = 4
    ADAPTIVE = 5


class SRecordType(Enum):
    HEADER = 0
    DATA = 3
    TERMINATION = 7


def read(data, end, includeEnd=False, skipFirst=False):
    readData = data.split(end)[0 if not skipFirst else 1]
    if includeEnd:
        readData += end

    return readData, len(readData) if not skipFirst else len(readData) + len(end)


In [ ]:
with open(FUL_FILE, "rb") as f:
    firmware = f.read()

if not firmware.startswith(b"\x1B%-12345X"):
    raise ValueError("Invalid ful file - Incorrect File Magic!")

pjlHeader, pjlHeaderLength = read(firmware, b"FWUPDATE!\x0d\x0a", True)

data = firmware[pjlHeaderLength:]

COMPRESSION_METHOD = CompressionMethod.UNENCODED
RASTER_WIDTH = 0

print(f"Starting to unpack {FUL_FILE}")
decompressedBuffer = bytearray()

totalDataLen = len(data)
offset = 0

while True:
    cmdData, cmdLength = read(data[offset:], b"\x1B", skipFirst=True)

    if cmdData.startswith(b"*rt"):  # *rt16384sA
        RASTER_WIDTH = int(cmdData[3:-2])
        # print(f"RASTER_WIDTH: {RASTER_WIDTH}")

        offset += cmdLength
        continue
    elif cmdData.startswith(b"*b"):
        if cmdLength < 10 and cmdData.endswith(b"Y"):
            # print("Ignoring: " + str(cmdData))
            offset += cmdLength
            continue

        # *b2m16362V{DATA}
        # *b16179V{DATA}
        cmdWithCompression = False
        if chr(cmdData[3]) == 'm':
            COMPRESSION_METHOD = CompressionMethod(int(chr(cmdData[2])))
            cmdWithCompression = True

        plane = True
        cmdWithoutData, cmdWithoutData_length = read(cmdData, b"V", includeEnd=True)

        try:
            compSize = int(cmdWithoutData[4 if cmdWithCompression else 2:-1])
        except:
            plane = False
            cmdWithoutData, cmdWithoutData_length = read(cmdData, b"W", includeEnd=True)
            try:
                compSize = int(cmdWithoutData[4 if cmdWithCompression else 2:-1])
            except:
                break

        compDataOffset = cmdWithoutData_length + 1  # +1 because of the \x1B byte

        compressedData = data[offset + compDataOffset:offset + compDataOffset + compSize]
        # print(f"Read {compSize} {COMPRESSION_METHOD.name} {'plane' if plane else 'row'} bytes")

        decompressedData = bytearray()

        if COMPRESSION_METHOD == CompressionMethod.UNENCODED:
            decompressedData.extend(compressedData)
        elif COMPRESSION_METHOD == CompressionMethod.TIFF:
            while True:
                if len(compressedData) == 0:
                    break

                control = unpack("b", compressedData[:1])[0]
                if control >= 0:
                    decompressedData.extend(compressedData[1:control + 2])
                    compressedData = compressedData[control + 2:]
                    continue
                elif control < 0:
                    if control == -128:
                        print("NOP")
                        exit(-1)
                        raise KeyboardInterrupt

                    repeat = abs(control)
                    decompressedData.extend(compressedData[1:2] * (repeat + 1))
                    compressedData = compressedData[2:]
        elif COMPRESSION_METHOD == CompressionMethod.DELTA_ROW:
            seed = bytearray()
            seed.extend(seed_row)

            seedLen = len(seed)
            seedOffset = 0

            if plane and seedLen < RASTER_WIDTH:
                count = (RASTER_WIDTH - seedLen)
                print(f"Zero filling {count} seed bytes")
                seed.extend(count * b"\x00")

            while True:
                if len(compressedData) == 0:
                    break

                command = unpack("B", compressedData[:1])[0]
                bytesToReplace = ((command & 0b11100000) >> 5) + 1
                deltaOffset = (command & 0b00011111)

                if deltaOffset == 31:
                    print("Unhandled 31 offset")
                    exit(-1)
                    raise KeyboardInterrupt

                deltaBytes = compressedData[1:1 + bytesToReplace]

                # print(f"command: {hex(command)}")
                # print(f"bytesToReplace: {bytesToReplace}")
                # print(f"deltaOffset: {deltaOffset}")
                # hexdump(deltaBytes)

                for i in range(bytesToReplace):
                    seed[seedOffset+deltaOffset+i] = deltaBytes[i]
                seedOffset += bytesToReplace + deltaOffset

                compressedData = compressedData[1 + bytesToReplace:]
            decompressedData.extend(seed)
        else:
            print(f"Unimplemented compression: {COMPRESSION_METHOD}")
            exit(-1)
            raise KeyboardInterrupt

        decompressedLength = len(decompressedData)
        if plane and decompressedLength < RASTER_WIDTH:
            count = (RASTER_WIDTH - decompressedLength)
            # print(f"Zero filling {count} bytes")
            decompressedData.extend(count * b"\x00")

        offset += compDataOffset + compSize
        decompressedBuffer.extend(decompressedData)

        seed_row = decompressedData
        continue
    else:
        print("Unhandled command!")
        hexdump(cmdData)
        raise KeyboardInterrupt

print("Finished first stage!")

with open(DECOMPRESSED_BUFFER, "wb") as decompressedbufferfile:
    decompressedbufferfile.write(decompressedBuffer)

In [ ]:
decompressedBuffer = bytearray()
with open(DECOMPRESSED_BUFFER, "rb") as decompressedbufferfile:
    decompressedBuffer.extend(decompressedbufferfile.read())

asciiSrecEndMatch = re.search(rb'([P])([A-F0-9a-f]{8})',decompressedBuffer) # From https://web.archive.org/web/20240526203007/https://www.jsof-tech.com/unpacking-hp-firmware-updates-part-2/

asciiSrecEnd = asciiSrecEndMatch.group(0)
print(asciiSrecEnd)
# asciiSrecEnd = b"F0047ACE9"
# asciiSrecEnd = b"F0041CCE9"

bootloaderSrec, binarySrec = decompressedBuffer.split(asciiSrecEnd)
bootloaderSrec += asciiSrecEnd
with open(BOOTLOADER_SREC, "wb") as bootloader:
    bootloader.write(bootloaderSrec)
print(f"Wrote {BOOTLOADER_SREC}")

with open(BINARY_SREC, "wb") as binarySrecFile:
    binarySrecFile.write(binarySrec)
print(f"Wrote {BINARY_SREC}")

In [ ]:
binarySrec = bytearray()
with open(BINARY_SREC, "rb") as binarySrecFile:
    binarySrec.extend(binarySrecFile.read())

bootloaderSrec = bytearray()
with open(BOOTLOADER_SREC, "rb") as bootloader:
    bootloaderSrec.extend(bootloader.read())


rawFlashBuffer = bytearray()

totalDataLen = len(binarySrec)
offset = 0

while True:
    srecStart = binarySrec[offset:offset + 3]
    if len(srecStart) == 1:
        break

    checksum, recTypeByte, recLength = unpack("BBB", srecStart)
    if (recTypeByte & 0xF0) != 0x30:
        print("Invalid binary SRecord type!")
        exit(-1)

    recordType = SRecordType(recTypeByte & 0xF)
    recLength -= 1

    if recordType == SRecordType.HEADER:
        address = binarySrec[offset + 3:offset + 5]
        recLength -= 2

        totalLength = 5 + recLength
        text = binarySrec[offset + 5:offset + totalLength]

        print("Binary SRecord header:")
        print(f"Address: {address.hex()}")
        print(f"Text: {text.decode()}")
        print()

        offset += totalLength
        continue
    elif recordType == SRecordType.DATA:
        address = binarySrec[offset + 3:offset + 7]
        recLength -= 4

        totalLength = 7 + recLength
        srecData = binarySrec[offset + 7:offset + totalLength]

        offset += totalLength
        rawFlashBuffer.extend(srecData)
        continue
    elif recordType == SRecordType.TERMINATION:
        startAddress = binarySrec[offset + 3:offset + 7]
        recLength -= 4

        print("Binary SRecord termination:")
        print(f"StartAddress: {startAddress.hex()}")
        print()

        offset += 7

print("Finished extracting raw flash image!")

with open(RAW_FLASH_BIN, "wb") as rawflash:
    rawflash.write(rawFlashBuffer)

In [ ]:

rawFlashBuffer = bytearray()
with open(RAW_FLASH_BIN, "rb") as rawflash:
    rawFlashBuffer.extend(rawflash.read())

print("Removing OOB data...")

flashBuffer = bytearray()

totalDataLen = len(rawFlashBuffer)
offset = 0

while True:
    print(f"Page at: {hex(offset)}")
    if offset >= totalDataLen:
        print(f"End offset: {hex(offset)}")
        break
    page = rawFlashBuffer[offset:offset + PAGE_SIZE]
    flashBuffer.extend(page)
    offset += PAGE_SIZE + OOB_SIZE

with open(FLASH_BIN, "wb") as flash:
    flash.write(flashBuffer)

print("Finished extracting flash image!")


In [ ]:
with open(CLEANED_BOOTLOADER_SREC,"w") as f2:
    with open(BOOTLOADER_SREC,"r") as f:
        unclean_srecs = f.readlines()
        f2.writelines([entry for i,entry in enumerate(unclean_srecs) if i>=7 and i<(len(unclean_srecs)-2)])

In [ ]:
bootloadersrecfile = bincopy.BinFile()
bootloadersrecfile.add_srec_file(CLEANED_BOOTLOADER_SREC)
print(bootloadersrecfile.info()) # print s19 as binary
for segment in bootloadersrecfile.segments:
    with open(BOOTLOADER_DUMPS/f"{hex(segment.address)}.bin","wb") as f:
        f.write(segment.data)

from capstone import *
# Disassemblers
disassemblers = {
    "ARM": Cs(CS_ARCH_ARM, CS_MODE_ARM),
    "Thumb": Cs(CS_ARCH_ARM, CS_MODE_THUMB),
}

for segment in bootloadersrecfile.segments:
    for name, md in disassemblers.items():
        print(f"\n--- {name} Mode ---")
        for i, insn in enumerate(md.disasm(segment.data, segment.address)):
            print(f"0x{insn.address:08x}:\t{insn.mnemonic}\t{insn.op_str}")
            if i >= 10:
                break


# with open(BOOTLOADER_DUMPS/"BootLoader.bin","wb") as f:
#     f.write(bootloadersrecfile.as_binary())

In [ ]:
bootloadersrecfile = bincopy.BinFile()
bootloadersrecfile.add_srec_file(CLEANED_BOOTLOADER_SREC)
print(bootloadersrecfile.info())

from capstone import *

entry_address = 0x201d947c
